In [1]:
import sys
sys.path.append('..')

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader

from src.models.basic_axial.basic_axialnet import BasicAxial
from src.models.basic_pga.basic_pga_net import BasicAxialPGA, OnlyPGA, BigOnlyPGA
from src.datasets.ice import Ice, IceWithProposals
from src.metrics.segmentation import _fast_hist, per_class_pixel_accuracy, jaccard_index, overall_pixel_accuracy, fw_miou
from src.metrics.utils import display_metric_list

import matplotlib.pyplot as plt

import os

In [2]:
def mean_iou(hist_matrix, class_names):
    classes = len(class_names)
    class_scores = np.zeros((classes))
    for i in range(classes):
        class_scores[i] = hist_matrix[i,i]/(max(1,np.sum(hist_matrix[i,:])+np.sum(hist_matrix[:,i])-hist_matrix[i,i]))
        print('class',class_names[i],'miou',class_scores[i])
    print('Mean IOU:',np.mean(class_scores))
    return class_scores

def freq_weighted_miou(hist_matrix, class_names):
    classes = len(class_names)
    class_scores = np.zeros((classes))
    for i in range(classes):
        class_scores[i] = np.sum(hist_matrix[i,:])*hist_matrix[i,i]/(max(1,np.sum(hist_matrix[i,:])))
    fmiou = np.sum(class_scores)/np.sum(hist_matrix).astype('float')
    print('Frequency Weighted mean accuracy:',fmiou)
    return fmiou

def hist_per_batch(tensor_1, tensor_2, ignore_label=255, classes=21):
    hist_tensor = torch.zeros(classes,classes)
    for class_2_int in range(classes):
        tensor_2_class = torch.eq(tensor_2,class_2_int).long()
        for class_1_int in range(classes):
            tensor_1_class = torch.eq(tensor_1,class_1_int).long()
            tensor_1_class = torch.mul(tensor_2_class,tensor_1_class)
            count = torch.sum(tensor_1_class)
            hist_tensor[class_2_int,class_1_int] +=count
    return hist_tensor

def mean_pixel_accuracy(hist_matrix, class_names):
    classes = len(class_names)
    class_scores = np.zeros((classes))
    for i in range(classes):
        class_scores[i] = hist_matrix[i,i]/(max(1,np.sum(hist_matrix[i,:])))
        print('class',class_names[i],'mean_pixel_accuracy',class_scores[i])
    return class_scores

def pixel_accuracy(hist_matrix):
    num = np.trace(hist_matrix)
    p_a =  num/max(1,np.sum(hist_matrix).astype('float'))
    print('Pixel accuracy:',p_a)
    return p_a

In [3]:
data_dir = '../data'
img_scale = .35
img_crop = 220

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# test_set = Ice(os.path.join(data_dir, 'imgs'), os.path.join(data_dir, 'masks'),
#                   os.path.join(data_dir, 'txt_files'), 'test', img_scale, img_crop)
# test_loader = DataLoader(test_set, batch_size=1)
train_set = IceWithProposals(os.path.join(data_dir, 'imgs'), os.path.join(data_dir, 'masks'),
                  os.path.join(data_dir, 'txt_files'), os.path.join(data_dir, 'proposals/binary_250_16'),
                               'train', img_scale, img_crop)

In [4]:
batch1 = train_set[1]
mask1 = batch1['mask']

batch2 = train_set[2]
mask2 = batch2['mask']

In [5]:
hist = _fast_hist(mask1, mask2, 3)

In [6]:
hist_new = hist_per_batch(mask1, mask2, classes=3)

In [7]:
hist

tensor([[21016., 11225.,  5683.],
        [ 2854.,  2237.,    94.],
        [ 2506.,  2077.,   708.]])

In [8]:
hist_new

tensor([[21016.,  2854.,  2506.],
        [11225.,  2237.,  2077.],
        [ 5683.,    94.,   708.]])

In [9]:
jaccard_index(hist)

(tensor(0.2235), tensor([0.4855, 0.1210, 0.0640]))

In [10]:
mean_iou(np.array(hist_new), ['ice1', 'ice2', 'water'])

class ice1 miou 0.4855373799800873
class ice2 miou 0.12100394815206528
class water miou 0.06396819651126862
Mean IOU: 0.22350317488114038


array([0.48553738, 0.12100395, 0.0639682 ])

In [11]:
freq_weighted_miou(np.array(hist_new), ['ice1', 'ice2', 'water'])

Frequency Weighted mean accuracy: 0.4950619834710744


0.4950619834710744

In [12]:
fw_miou(hist)

tensor(0.4951)

In [13]:
hist.T

tensor([[21016.,  2854.,  2506.],
        [11225.,  2237.,  2077.],
        [ 5683.,    94.,   708.]])

In [14]:
hist_tmp = hist.T
class_names = ['ice1', 'ice2', 'water']

classes = len(class_names)
class_scores = torch.zeros((classes))
for i in range(classes):
    class_scores[i] = torch.sum(hist_tmp[i,:])*hist_tmp[i,i]/(max(1,torch.sum(hist_tmp[i,:])))
fmiou = torch.sum(class_scores)/torch.sum(hist_tmp)#.astype('float')
print('Frequency Weighted mean accuracy:',fmiou)

Frequency Weighted mean accuracy: tensor(0.4951)


In [15]:
len(hist)

3

In [16]:
torch.zeros((3))

tensor([0., 0., 0.])

In [17]:
per_class_pixel_accuracy(hist)

(tensor(0.3731), tensor([0.5542, 0.4314, 0.1338]))

In [22]:
overall_pixel_accuracy(hist)

0.495061993598938

In [19]:
mean_pixel_accuracy(np.array(hist_new), ['ice1', 'ice2', 'water'])

class ice1 mean_pixel_accuracy 0.7967849373817444
class ice2 mean_pixel_accuracy 0.1439603567123413
class water mean_pixel_accuracy 0.10917501896619797


array([0.79678494, 0.14396036, 0.10917502])

In [20]:
pixel_accuracy(np.array(hist_new))

Pixel accuracy: 0.4950619834710744


0.4950619834710744